<a href="http://landlab.github.io"><img style="float: left" src="../../../landlab_header.png"></a>

# Using the SteepnessFinder Component

## Background

Given an input digital elevation model (DEM), the `SteepnessFinder` component calculates the **steepness index** for nodes or stream segments in the drainage network. The steepness index is measure of channel gradient that is normalized to compensate for the correlation between gradient and drainage area. The definition of the steepness index derives from an idealized mathematical relationship between channel gradient and drainage area,

$$S = k_{sn} A^\theta$$

where $S$ is local channel gradient, $A$ is drainage area, $k_{sn}$ is the steepness index, and $\theta$ is the **concavity index** (because its value reflects the upward concavity of the stream profile; a value of 0 would represent a linear profile with no concavity). The definition of steepness index is therefore

$$k_{sn} = \frac{S}{A^\theta}$$

The occurrence of an approximate power-law relationship between gradient and drainage area was noted by, for example, Hack (1957, his equation 2) and Flint (1974) (it is sometimes called "Flint's Law", John Hack having already had a different scaling relation named for him; see the *HackCalculator* component tutorial). The emergence of DEMs and computers powerful enough to analyze them opened the door to statistical exploration of the slope-area relation (Tarboton and Bras, 1989), and the recognition that the relationship can be interpreted in terms of geomorphic processes (Willgoose et al., 1991). The concavity and steepness indices are defined and discussed in Whipple and Tucker (1999). The steepness index, and the related metric the chi index (see the *ChiFinder* tutorial) have become widely used as methods for identifying anomalies in channel gradient that may related to tectonics, lithology, or landscape transience (see, e.g., Wobus et al., 2006; Kirby and Whipple, 2012).

## Imports

First, import what we'll need:

In [ ]:
import copy

import matplotlib as mpl
import numpy as np

from landlab import imshow_grid
from landlab.components import FlowAccumulator, SteepnessFinder
from landlab.io import read_esri_ascii

## Documentation

The [Reference Documentation](https://landlab.readthedocs.io/en/latest/reference/components/index.html#landlab.components.SteepnessFinder) provides information about the SteepnessFinder class, describes its methods and attributes, and provides a link to the source code.

The SteepnessFinder class docstring describes the component and provides some simple examples:

In [ ]:
print(SteepnessFinder.__doc__)

The `__init__` docstring lists the parameters:

In [ ]:
print(SteepnessFinder.__init__.__doc__)

## Example 1

In this example, we read in a small digital elevation model (DEM) from the Sevilleta National Wildlife Refuge, NM, USA.

The DEM file is in ESRI Ascii format, with NODATA codes for cells outside the main watershed. We'll use the Landlab grid method `set_watershed_boundary_condition` to assign closed-boundary status to NODATA cells.

In [ ]:
# read the DEM
(grid, elev) = read_esri_ascii("hugo_site_filled.asc", name="topographic__elevation")
grid.set_watershed_boundary_condition(elev)

In [ ]:
cmap = copy.copy(mpl.colormaps["pink"])
imshow_grid(grid, elev, cmap=cmap, colorbar_label="Elevation (m)")

The `SteepnessFinder` needs to have drainage areas pre-calculated. We'll do that with the `FlowAccumulator` component. We'll have the component do D8 flow routing (each DEM cell drains to whichever of its 8 neighbors lies in the steepest downslope direction), and fill pits (depressions in the DEM that would otherwise block the flow) using the `LakeMapperBarnes` component. The latter two arguments below tell the lake mapper to update the flow directions and drainage areas after filling the pits.

In [ ]:
fa = FlowAccumulator(
    grid,
    flow_director="FlowDirectorD8",  # use D8 routing
)
fa.run_one_step()  # run the flow accumulator
cmap = copy.copy(mpl.colormaps["Blues"])
imshow_grid(
    grid,
    np.log10(grid.at_node["drainage_area"] + 1.0),  # sq root helps show drainage
    cmap=cmap,
    colorbar_label="Log10(drainage area (m2))",
)

Now run the `SteepnessFinder` and display the map of $k_{sn}$ values:

In [ ]:
sf = SteepnessFinder(grid, min_drainage_area=2.0e4)
sf.calculate_steepnesses()
cmap = copy.copy(mpl.colormaps["viridis"])
imshow_grid(
    grid,
    grid.at_node["channel__steepness_index"],
    cmap=cmap,
    colorbar_label="Steepness index",
)

## Example 2: fixed elevation drop

One challenge in extracting $k_{sn}$ from digital elevation data is noise: cell-to-cell variations in slope can make it hard to visualize coherent patterns, as the above example demonstrates. One solution, discussed by Wobus et al. (2006), is to use a fixed elevation drop: one starts from a given pixel and iterates from pixel-to-pixel downstream until the elevation difference from the starting point is equal to or greater than a specified drop distance. One advantage of this method prevents the analyzed segments from having zero slope. Another is that it effectively averages gradient over a longer horizontal distance that depends on the local gradient: lower gradients, which are generally more prone to noise, will be averaged over a longer distance, and vice versa. The example below shows how to do this with SteepnessFinder:

In [ ]:
# calculate steepness
sf = SteepnessFinder(grid, elev_step=4.0, min_drainage_area=2.0e4)
sf.calculate_steepnesses()
cmap = copy.copy(mpl.colormaps["viridis"])
imshow_grid(
    grid,
    grid.at_node["channel__steepness_index"],
    cmap=cmap,
    colorbar_label="Steepness index",
)

## References

Flint, J. J. (1974). Stream gradient as a function of order, magnitude, and discharge. Water Resources Research, 10(5), 969-973.

Hack, J. T. (1957). Studies of longitudinal stream profiles in Virginia and Maryland. Geological Survey Professional Paper 294-B. US Government Printing Office.

Kirby, E., & Whipple, K. X. (2012). Expression of active tectonics in erosional landscapes. Journal of Structural Geology, 44, 54-75.

Tarboton, D. G., Bras, R. L., & Rodriguez‐Iturbe, I. (1989). Scaling and elevation in river networks. Water Resources Research, 25(9), 2037-2051.

Whipple, K. X., & Tucker, G. E. (1999). Dynamics of the stream‐power river incision model: Implications for height limits of mountain ranges, landscape response timescales, and research needs. Journal of Geophysical Research: Solid Earth, 104(B8), 17661-17674.

Willgoose, G., Bras, R. L., & Rodriguez‐Iturbe, I. (1991). A physical explanation of an observed link area‐slope relationship. Water Resources Research, 27(7), 1697-1702.

Wobus, C. W., Whipple, K. X., Kirby, E., Snyder, N. P., Johnson, J., Spyropolou, K., Crosby, B. T., and Sheenan, D.: Tectonics from topography: Procedures, promise, and pitfalls, in: Tectonics, Climate, and Landscape Evolution, edited by: Willett, S. D., Hovius, N., Brandon, M. T., and Fisher, D., Geological Society of America Special Paper 398, Geological Society of America, Boulder, CO, USA, 55–74, 2006.

For more Landlab tutorials, click here: https://landlab.readthedocs.io/en/latest/user_guide/tutorials.html